# Lets Run a Hybrid Search

In this notebook we'll learn about how to use a ML model to enrich our Icecat data with vectors, and then we'll balance a lexical search with a neural search.

In [1]:
import requests
import json
import mercury as mr

app = mr.App(title="Let's Run a Hybrid Search", static_notebook=True)

mercury.App

## Configure ml plugin

In [2]:
url = "http://localhost:9200/_cluster/settings"
headers = {
    'Content-Type': 'application/json'
}

payload = {
    "persistent": {
        "plugins": {
            "ml_commons": {
                "only_run_on_ml_node": "false",
                "model_access_control_enabled": "true",
                "native_memory_threshold": "99"
            }
        }
    }
}


response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))



In [3]:
mr.JSON(response.json(), level=4)

## Register a model group

In [4]:
url = "http://localhost:9200/_plugins/_ml/model_groups/_register"

payload = {
  "name": "NLP_model_group",
  "description": "A model group for NLP models"
}


response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)

model_group_id = response.json()['model_group_id']

print(f"Created Model Group ID {model_group_id}")

Created Model Group ID 8LNIlZEBb9vhxCYTfNkQ


## Register the model to the model group


In [6]:
url = "http://localhost:9200/_plugins/_ml/models/_register"

payload = {
  "name": "huggingface/sentence-transformers/msmarco-distilbert-base-tas-b",
  "version": "1.0.1",
  "model_group_id": model_group_id,
  "model_format": "TORCH_SCRIPT"
}


response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)
task_id = response.json()['task_id']

## Check status of registering model

In [10]:
url = f"http://localhost:9200/_plugins/_ml/tasks/{task_id}"

response = requests.request("GET", url, headers=headers)
mr.JSON(response.json(), level=4)

model_id = response.json()['model_id']

print(f"Created Model ID {model_id}")

## Deploy the model

In [11]:
url = f"http://localhost:9200/_plugins/_ml/models/{model_id}/_deploy"

response = requests.request("POST", url, headers=headers)
mr.JSON(response.json(), level=4)
deploy_model_task_id = response.json()['task_id']

## Check the status

In [12]:
url = f"http://localhost:9200/_plugins/_ml/tasks/{deploy_model_task_id}"

response = requests.request("GET", url, headers=headers)
mr.JSON(response.json(), level=4)

model_id = response.json()['model_id']

## Create an ingest pipeline

In [13]:
url = "http://localhost:9200/_ingest/pipeline/nlp-ingest-pipeline"

payload = {
  "description": "A text embedding pipeline",
  "processors": [
    {
      "text_embedding": {
        "model_id": model_id,
        "field_map": {
          "passage_text": "passage_embedding"
        }
      }
    }
  ]
}


response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)


## Create an index for ingestion

In [16]:
url = "http://localhost:9200/my-nlp-index"

response = requests.request("DELETE", url, headers=headers)

payload = {
  "settings": {
    "index.knn": True,
    "default_pipeline": "nlp-ingest-pipeline"
  },
  "mappings": {
    "properties": {
      "id": {
        "type": "text"
      },
      "passage_embedding": {
        "type": "knn_vector",
        "dimension": 768,
        "method": {
          "engine": "lucene",
          "space_type": "l2",
          "name": "hnsw",
          "parameters": {}
        }
      },
      "passage_text": {
        "type": "text"
      }
    }
  }
}
response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)

## Ingest documents into the index

#### Document 1

In [20]:
url = "http://localhost:9200/my-nlp-index/_doc/1"

payload = {
  "passage_text": "Hello world",
  "id": "s1"
}


response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=1)

#### Document 2

In [19]:
url = "http://localhost:9200/my-nlp-index/_doc/2"

payload = {
  "passage_text": "Hi planet",
  "id": "s2"
}


response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=1)

## Configure a search pipeline

In [21]:
url = "http://localhost:9200/_search/pipeline/nlp-search-pipeline"

payload = {
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              0.3,
              0.7
            ]
          }
        }
      }
    }
  ]
}


response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)

## Search the index using just Neural search

In [29]:
url = "http://localhost:9200/my-nlp-index/_search"

payload = {
  "_source": {
    "exclude": [
      "passage_embedding"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "neural": {
            "passage_embedding": {
              "query_text": "Hi world",
              "model_id": model_id,
              "k": 5
            }
          }
        }
      ]
    }
  }
}


response = requests.request("GET", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=5)

## Combining neural and lexical match query

Notice that we add to the mix both a lexical (_match_) and a neural query.  They are blended by the `nlp-search-pipeline` that we reference.

In [31]:
url = "http://localhost:9200/my-nlp-index/_search?search_pipeline=nlp-search-pipeline"

payload = {
  "_source": {
    "exclude": [
      "passage_embedding"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "passage_text": {
              "query": "Hi world"
            }
          }
        },
        {
          "neural": {
            "passage_embedding": {
              "query_text": "Hi world",
              "model_id": model_id,
              "k": 5
            }
          }
        }
      ]
    }
  }
}


response = requests.request("GET", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=5)